### Capstone Project

In [1]:
import os 
import pandas as pd
import plotly.express as plt
from datetime import timedelta
from datetime import datetime

### Step-1: Load Data by Path

In [2]:
Root_path = 'C:\\Users\\mdmes\\OneDrive\\Desktop\\Pandas Data'

product_path = os.path.join(Root_path,'products.csv')
purchases_path =os.path.join(Root_path, 'purchases.csv')
sales_path = os.path.join(Root_path, 'sales.csv')


In [3]:
product_data = pd.read_csv(
    product_path
)
product_data.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level
0,1001,Organic Rice,Groceries,1551.26,2296.31,43
1,1002,Olive Oil,Groceries,2516.38,3370.60,39
2,1003,Canned Beans,Groceries,961.25,1117.36,30
3,1004,Pasta,Groceries,696.83,1007.94,36
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27


In [4]:
purchases_data = pd.read_csv(
    purchases_path
)

purchases_data.head()

,product_id,quantity_purchased,purchase_date
0,1095,94,2024-08-19
1,1054,23,2024-08-16
2,1058,64,2024-12-18
3,1067,57,2024-06-12
4,1046,91,2024-03-13


In [5]:
sales_data = pd.read_csv(
    sales_path
)

sales_data.head()

,product_id,quantity_sold,sale_date,location,discount
0,1013,4,2024-09-02,Rajshahi,52.178882
1,1064,9,2024-02-25,Sylhet,52.059982
2,1011,2,2024-02-07,Dhaka,7.370982
3,1019,2,2024-09-01,Sylhet,51.219448
4,1011,9,2024-01-21,Chittagong,82.274403


### Step-2:Convert string to datetime 

In [6]:
sales_data['sale_date'] = pd.to_datetime(sales_data['sale_date']).dt.date

date1 =datetime.strptime('2024-09-01','%Y-%m-%d').date()
date2 =datetime.strptime('2024-09-03','%Y-%m-%d').date()

date = sales_data['sale_date'].iloc[0]

date1 <= date <= date2

True

In [7]:
purchases_data['purchase_date'] = pd.to_datetime(purchases_data['purchase_date']).dt.date

date3 = datetime.strptime('2024-08-18','%Y-%m-%d').date()
date4 = datetime.strptime('2024-08-20','%Y-%m-%d').date()
Date = purchases_data['purchase_date'].iloc[0]

date3 <= Date <= date4

True

### Step-3: Compute Current Stock

In [8]:
product_id = 1095

total_purchase = purchases_data[
    purchases_data['product_id'] == product_id]['quantity_purchased'
    ].sum()

total_sale = sales_data[
    sales_data['product_id'] == product_id]['quantity_sold'
    ].sum()

total_stock = total_purchase - total_sale

total_stock

np.int64(476)

In [9]:
def Current_Stock(product_id):
    # per product purchase quantity
    per_product_purchase = purchases_data[
        purchases_data['product_id'] == product_id]['quantity_purchased'
    ].sum()

    # per product sales quantity
    per_product_sale = sales_data[
        sales_data['product_id'] == product_id]['quantity_sold'
    ].sum()


    # Current Stock
    per_product_stock = per_product_purchase - per_product_sale

    return per_product_stock

Current_Stock(1095)

np.int64(476)

In [10]:
product_data['Current_Stock'] = product_data['product_id'].apply(
    lambda product_id : Current_Stock(
        product_id=product_id
        )
)

product_data.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,Current_Stock
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120
3,1004,Pasta,Groceries,696.83,1007.94,36,204
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575


### Step-4: Compute per product Profit

In [11]:
def per_product_profit(product_id):
    per_product_sold_quantity = sales_data[
        sales_data['product_id'] == product_id
    ]['quantity_sold'].sum()

    product = product_data[product_data['product_id'] == product_id]

    per_product_profit = product['selling_price'] - product['cost_price']
    per_product_profit = per_product_profit.iloc[0]

    per_product_total_profit = per_product_profit * per_product_sold_quantity

    return per_product_total_profit

per_product_profit(1095)

product_data['Per_Product_Profit'] = product_data['product_id'].apply(
    lambda product_id : per_product_profit(
        product_id=product_id
        )
)

product_data.head()


,product_id,product_name,category,cost_price,selling_price,reorder_level,Current_Stock,Per_Product_Profit
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658,158695.65
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301,146071.62
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120,33407.54
3,1004,Pasta,Groceries,696.83,1007.94,36,204,101732.97
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575,134286.66


### Step-5:Find the slow moving Products

In [12]:
startDate = datetime.strptime('2024-12-31','%Y-%m-%d').date()
lastDate = startDate - timedelta(days=90)
product_id = 1095
product_Sale_by_90days = sales_data[ 
    (sales_data['product_id'] == product_id)&
    (sales_data['sale_date'] >= lastDate)
]['quantity_sold'].sum()

def Slow_Moving_Product(product_id):
    per_product_info = sales_data[ 
        (sales_data['product_id'] == product_id) &
        (sales_data['sale_date'] >= lastDate)
    ]['quantity_sold'].sum()

    return per_product_info < 40

product_data['Slow_Moving_Products'] = product_data['product_id'].apply(
    lambda product_id: Slow_Moving_Product(
        product_id=product_id
        )
)

product_data.head()


,product_id,product_name,category,cost_price,selling_price,reorder_level,Current_Stock,Per_Product_Profit,Slow_Moving_Products
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658,158695.65,False
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301,146071.62,True
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120,33407.54,False
3,1004,Pasta,Groceries,696.83,1007.94,36,204,101732.97,False
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575,134286.66,True


### Step-6: Find Over and Under stock

In [13]:
def Stock_status(product_id):
    product = product_data[
        product_data['product_id'] == product_id
    ].iloc[0]

    current_stock = product['Current_Stock']
    recorder_level = product['reorder_level']

    if current_stock < recorder_level: return 'UnderStock'
    elif current_stock > recorder_level*15: return 'OverStock'
    return 'Perfect Stock'


Stock_status(1095)

product_data['Stock_Status'] = product_data['product_id'].apply(
    lambda product_id: Stock_status(
        product_id=product_id
    )
)

product_data.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,Current_Stock,Per_Product_Profit,Slow_Moving_Products,Stock_Status
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658,158695.65,False,OverStock
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301,146071.62,True,Perfect Stock
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120,33407.54,False,Perfect Stock
3,1004,Pasta,Groceries,696.83,1007.94,36,204,101732.97,False,Perfect Stock
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575,134286.66,True,OverStock


### Step-7: Per Product Revenue

In [14]:
def per_product_revenue(product_data, sales_data, product_id):
    selling_price = product_data[ 
        product_data['product_id'] == product_id
    ]['selling_price'].iloc[0]

    quantity_sold = sales_data[ 
        sales_data['product_id'] == product_id
    ]['quantity_sold'].sum()

    revenue = quantity_sold * selling_price

    return revenue

per_product_revenue(
    product_data=product_data,
    sales_data=sales_data,
    product_id=1095
)



np.float64(13319087.8)

### Step-8: Queries Business data

In [15]:
def sales_between_dates(sales_data, startDate, lastDate, location):
    return sales_data[ 
        (sales_data['sale_date'] >= startDate)&
        (sales_data['sale_date'] <= lastDate)&
        (sales_data['location'].isin(location))
    ]

In [16]:
def Selected_category(product_data, category):
    return product_data[product_data['category'].isin(category)]


In [17]:
def Product_UnderStock(product_data):
    return product_data[product_data['Stock_Status'] == 'UnderStock']

In [18]:
def summay_of_data(product_data, sales_data):
    total_revenue = product_data['product_id'].apply(
        lambda product_id: per_product_revenue(
            product_data=product_data,
            product_id=product_id,
            sales_data=sales_data
        )
    ).sum()

    total_profit = product_data['Per_Product_Profit'].sum()
    total_sold_quantity = sales_data['quantity_sold'].sum()
    total_understock = len(Product_UnderStock(
        product_data=product_data
    ))


    return {
        'Total Revenue (K)':int(total_revenue/1e3),
        'Total Profit (K)':int(total_profit/1e3),
        'Total Sold Quantity (K)' : int(total_sold_quantity/1e3),
        'Total Understock Product ':total_understock
    }

In [19]:
Start_date  = datetime.strptime('2024-01-01','%Y-%m-%d').date()
Last_date  = datetime.strptime('2024-12-31','%Y-%m-%d').date()

category_select = ['Groceries']
location_select = ['Dhaka']

In [20]:
filtered_sales = sales_between_dates(
    sales_data=sales_data,
    startDate=Start_date,
    lastDate=Last_date,
    location=location_select
)
filtered_sales.head()

,product_id,quantity_sold,sale_date,location,discount
2,1011,2,2024-02-07,Dhaka,7.370982
5,1077,7,2024-04-19,Dhaka,94.679928
8,1001,6,2024-08-18,Dhaka,56.834725
9,1040,7,2024-01-08,Dhaka,89.031249
11,1076,6,2024-10-16,Dhaka,96.526187


In [21]:
select_category_func=Selected_category(
    product_data=product_data,
    category=category_select
)
select_category_func.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,Current_Stock,Per_Product_Profit,Slow_Moving_Products,Stock_Status
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658,158695.65,False,OverStock
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301,146071.62,True,Perfect Stock
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120,33407.54,False,Perfect Stock
3,1004,Pasta,Groceries,696.83,1007.94,36,204,101732.97,False,Perfect Stock
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575,134286.66,True,OverStock


In [22]:
select_understock_func = Product_UnderStock(
    product_data=product_data
)

select_understock_func.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,Current_Stock,Per_Product_Profit,Slow_Moving_Products,Stock_Status
10,1011,Wireless Mouse,Electronics,54960.08,70552.06,22,-30,3570563.42,False,UnderStock
27,1028,Sunglasses,Clothing,6891.59,13406.29,20,-35,2019557.00,False,UnderStock
30,1031,Fresh Salmon,Perishables,1181.47,1352.76,33,22,29804.46,False,UnderStock
47,1048,Almond Milk,Groceries,2621.61,3401.59,33,-47,180955.36,False,UnderStock
58,1059,Fitness Tracker,Electronics,70497.44,92488.55,40,-101,8774452.89,False,UnderStock


### Step-9: Visualaization by plot

In [23]:
top_products = select_category_func.nlargest(10,'Per_Product_Profit')[['product_name', 'Per_Product_Profit']]
print(top_products)

plot1 = plt.bar(top_products, x='product_name', y='Per_Product_Profit',title='Top 10 Products')
plot1.show()

    product_name  Per_Product_Profit
44  Coffee Beans           246773.82
46  Granola Bars           241602.34
81     Olive Oil           200182.88
43         Pasta           197819.76
40  Organic Rice           192514.56
47   Almond Milk           180955.36
0   Organic Rice           158695.65
80  Organic Rice           153655.36
1      Olive Oil           146071.62
48         Honey           142789.50


In [26]:
category_profit = select_category_func.groupby('category')['Per_Product_Profit'].sum().reset_index()
plot2=plt.pie(
    category_profit,
    values='Per_Product_Profit',
    names='category',
    title='Profit Distribution by category'
)

plot2.show()

### Step-10: Tables

In [25]:
Summary_information = select_category_func[['product_name','category','Current_Stock','reorder_level','Per_Product_Profit','Stock_Status']]

Summary_information.head()

,product_name,category,Current_Stock,reorder_level,Per_Product_Profit,Stock_Status
0,Organic Rice,Groceries,658,43,158695.65,OverStock
1,Olive Oil,Groceries,301,39,146071.62,Perfect Stock
2,Canned Beans,Groceries,120,30,33407.54,Perfect Stock
3,Pasta,Groceries,204,36,101732.97,Perfect Stock
4,Coffee Beans,Groceries,575,27,134286.66,OverStock
